# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon, make_map
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from regions import boolean_mask, SOM_area, Nino12, Nino34, global_ocean,\
                    gl_ocean_rect, NPacific_mask_rect,\
                    Nino12_low, Nino34_low, TexT_mask, AMO_mask, SST_index_bounds
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev
from xr_DataArrays import xr_AREA
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend,\
                          ocn_field_regression, lag_linregress_3D

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' )
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' )

# SST indices

In [ ]:
MASK_ocn  = boolean_mask('ocn'     , 0)
MASK_low  = boolean_mask('ocn_low' , 0)
MASK_rect = boolean_mask('ocn_rect', 0)

In [ ]:
TAREA       = xr_AREA('ocn')
AREA_low    = xr_AREA('ocn_low')
AREA_rect   = xr_AREA('ocn_rect')

# Atlantic Ocean

## Atlantic Multidecadal Oscillation
from Wikipedia:
> Several methods have been proposed to remove the global trend and El Niño-Southern Oscillation (ENSO) influence over the North Atlantic SST. Trenberth and Shea, assuming that the effect of global forcing over the North Atlantic is similar to the global ocean, subtracted the global (60°N-60°S) mean SST from the North Atlantic SST to derive a revised AMO index.[6]
>
> Ting et al. however argue that the forced SST pattern is not globally uniform; they separated the forced and internally generated variability using signal to noise maximizing EOF analysis.[2]
>
> Van Oldenborgh et al. derived an AMO index as the SST averaged over the extra-tropical North Atlantic (to remove the influence of ENSO that is greater at tropical latitude) minus the regression on global mean temperature.[7]
>
> Guan and Nigam removed the non stationary global trend and Pacific natural variability before applying an EOF analysis to the residual North Atlantic SST.[8]
>
> The linearly detrended index suggests that the North Atlantic SST anomaly at the end of the twentieth century is equally divided between the externally forced component and internally generated variability, and that the current peak is similar to middle twentieth century; by contrast the others methodology suggest that a large portion of the North Atlantic anomaly at the end of the twentieth century is externally forced.[2]

We use the method of Trenberth here: averaged SST [0-60 N]x[0-80W] - avg. SST [60S-60N], then .

In [ ]:
# plotting tropics-extratropics and North atlantic maps
f,ax = plt.subplots(2,2, figsize=(8,5), sharey='row')
for i, domain in enumerate(['ocn', 'ocn_low']):
    xr_AREA(domain).where(TexT_mask(domain)).plot(ax=ax[i,0])
    xr_AREA(domain).where(AMO_mask(domain)).plot(ax=ax[i,1])
plt.tight_layout()

In [ ]:
def AMO(run):
    if run in ['ctrl', 'rcp']:
        domain    = 'ocn'
        AREA      = TAREA
        MASK      = MASK_ocn
    elif run in ['lpi', 'lpd']:
        domain    = 'ocn_low'
        AREA      = AREA_low
        MASK      = MASK_low
        
    if run =='ctrl':   SST_yrly = SST_yrly_ctrl
    elif run =='rcp':  SST_yrly = SST_yrly_rcp
    elif run =='lpd':  SST_yrly = SST_yrly_lpd
    elif run =='lpi':  SST_yrly = SST_yrly_lpi
        
    dims = ('nlat', 'nlon')
    AREA_AMO  = AREA.where(AMO_mask(domain)).sum()
    AREA_TexT = AREA.where(TexT_mask(domain)).sum()
    print(AREA_AMO.values, AREA_TexT.values)
    
    NA   = SST_index(xa_SST=SST_yrly, AREA=AREA, index_loc=None, AREA_index=AREA_AMO , MASK=AMO_mask(domain) , dims=dims)
    TexT = SST_index(xa_SST=SST_yrly, AREA=AREA, index_loc=None, AREA_index=AREA_TexT, MASK=TexT_mask(domain), dims=dims)
    
    AMO  = (NA - TexT) - (NA - TexT).mean()
    AMO.to_netcdf(f'{path_results}/SST/AMO_yrly_{run}.nc')

    f,ax = plt.subplots(3,1,figsize=(8,5),sharex=True)
    ax[0].plot(NA)
    ax[1].plot(TexT)
    ax[2].plot(AMO)
    plt.tight_layout()
    return AMO

In [ ]:
# %%time
# AMO_ctrl = AMO('ctrl')  # 1:20
# AMO_rcp  = AMO('rcp' )  # 40 sec
# AMO_lpd  = AMO('lpd' )  # 4 sec
# AMO_lpi  = AMO('lpi' )  # 9 sec

In [ ]:
AMO_ctrl = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_ctrl.nc', decode_times=False)
AMO_rcp  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_rcp.nc' , decode_times=False)
AMO_lpd  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_lpd.nc' , decode_times=False)
AMO_lpi  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_lpi.nc' , decode_times=False)

In [ ]:
# comparing filtering methods and timescales
plt.figure(figsize=(8,5))
plt.axhline(0,c='k', lw=.5)
plt.plot(AMO_ctrl)
plt.plot(AMO_ctrl.rolling(time=10, center=True).mean())
plt.plot(AMO_ctrl.rolling(time=40, center=True).mean())
plt.plot(lowpass(AMO_ctrl.values, 40))
plt.plot(lowpass(AMO_ctrl.values, 10))

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.plot([2050]*2, [-.15,.2], c='g', lw=.5)
ax.plot([2200]*2, [-.15,.2], c='g', lw=.5)

L1, = ax.plot(AMO_ctrl.time/365+1850, chebychev(AMO_ctrl.values, 13), c='C0', ls='--', lw=1, label='chebychev 13 yr')
ax.plot(      AMO_rcp .time/365+ 200, chebychev(AMO_rcp .values, 13), c='C1', ls='--', lw=1)
ax.plot(      AMO_lpd .time/365+1350, chebychev(AMO_lpd .values, 13), c='C2', ls='--', lw=1)
ax.plot(      AMO_lpi .time/365-1600, chebychev(AMO_lpi .values, 13), c='C3', ls='--', lw=1)

L2, = ax.plot(AMO_ctrl.time/365+1850, lowpass(AMO_ctrl.values, 40), c='C0', ls='-' , lw=2, label='low pass 40 yr')
ax.plot(      AMO_rcp .time/365+ 200, lowpass(AMO_rcp .values, 40), c='C1', ls='-' , lw=2)
ax.plot(      AMO_lpd .time/365+1350, lowpass(AMO_lpd .values, 40), c='C2', ls='-' , lw=2)
ax.plot(      AMO_lpi .time/365-1600, lowpass(AMO_lpi .values, 40), c='C3', ls='-' , lw=2)

L3, = ax.plot(AMO_ctrl.time/365+1850, xr_lintrend(chebychev(AMO_ctrl,13)), c='C0', lw=1.5, ls='-.', label='linear trend')
ax.plot(      AMO_rcp .time/365+ 200, xr_lintrend(chebychev(AMO_rcp ,13)), c='C1', lw=1.5, ls='-.')
ax.plot(      AMO_lpd .time/365+1350, xr_lintrend(chebychev(AMO_lpd ,13)), c='C2', ls='-.' , lw=1.5)
ax.plot(      AMO_lpi .time/365-1600, xr_lintrend(chebychev(AMO_lpi ,13)), c='C3', ls='-.' , lw=1.5)

ax.text(1950, .21, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .21, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .21, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .21, 'pre-ind. low' , fontsize=16, color='C3')

ax.legend(handles=[L1, L2, L3], loc=8, ncol=3, fontsize=14, frameon=False)
ax.set_xlabel('time[years]', fontsize=16)
ax.set_ylabel('AMO index [$^\circ$C]'  , fontsize=16)
ax.set_ylim((-.17,.24))
ax.set_xticks(np.arange(0,2400,200))
f.tight_layout()
plt.savefig(f'{path_results}/SST/AMO_index_overview')

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.axvline(200,c='g', lw=.5)
ax.axvline(300,c='g', lw=.5)
L1, = ax.plot(AMO_ctrl.time/365     , AMO_ctrl, c='C0', lw=.5, ls=':', label='yrly raw data')
ax.plot(AMO_rcp .time/365-1700, AMO_rcp , c='C1', lw=.5, ls=':')
# ax.plot(AMO_ctrl.time/365     , AMO_ctrl.rolling(time=15, center=True).mean(), c='C0', ls='-.', lw=1.5, label='15 yr running mean')
# ax.plot(AMO_rcp .time/365-1700, AMO_rcp .rolling(time=15, center=True).mean(), c='C1', ls='-.', lw=1.5)
L2, = ax.plot(AMO_ctrl.time/365     , lowpass(AMO_ctrl.values, 15), c='C0', lw=1.5, ls='--', label='low pass 15 yr')
ax.plot(AMO_rcp .time/365-1700, lowpass(AMO_rcp .values, 15), c='C1', lw=1.5, ls='--')
L3, = ax.plot(AMO_ctrl.time/365     , lowpass(AMO_ctrl.values, 40), c='C0', lw=2, label='low pass 40 yr')
ax.plot(AMO_rcp .time/365-1700, lowpass(AMO_rcp .values, 40), c='C1', lw=2)
L4, = ax.plot(AMO_ctrl.time/365     , xr_lintrend(AMO_ctrl), c='C0', lw=1.5, ls='-.', label='linear trend')
ax.plot(AMO_rcp .time/365-1700, xr_lintrend(AMO_rcp ), c='C1', lw=1.5, ls='-.')

ax.text(.05, .92, 'CTRL', transform=ax.transAxes, fontsize=16, color='C0')
ax.text(.72, .92, 'RCP', transform=ax.transAxes, fontsize=16, color='C1')
ax.legend(handles=[L1, L2, L3, L4], loc=3, ncol=4, fontsize=14, frameon=False)
ax.set_xlabel('time [years]' , fontsize=16)
ax.set_ylabel('AMO index [K]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/SST/AMO_index_ctrl_rcp')

In [ ]:
# filtered and detrended AMO indices
AMO_filt_detr_ctrl = chebychev(AMO_ctrl, 13)-xr_lintrend(chebychev(AMO_ctrl,13))
AMO_filt_detr_rcp  = chebychev(AMO_rcp , 13)-xr_lintrend(chebychev(AMO_rcp ,13))
AMO_filt_detr_lpd  = chebychev(AMO_lpd , 13)-xr_lintrend(chebychev(AMO_lpd ,13))
AMO_filt_detr_lpi  = chebychev(AMO_lpi , 13)-xr_lintrend(chebychev(AMO_lpi ,13))
AMO_filt_detr_ctrl.to_netcdf(f'{path_results}/SST/AMO_filt_detr_ctrl.nc')
AMO_filt_detr_rcp .to_netcdf(f'{path_results}/SST/AMO_filt_detr_rcp.nc' )
AMO_filt_detr_lpd .to_netcdf(f'{path_results}/SST/AMO_filt_detr_lpd.nc' )
AMO_filt_detr_lpi .to_netcdf(f'{path_results}/SST/AMO_filt_detr_lpi.nc' )

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.plot([2050]*2, [-.15,.13], c='g', lw=.5)
ax.plot([2200]*2, [-.15,.13], c='g', lw=.5)

L1, = ax.plot(AMO_ctrl.time/365+1850, AMO_filt_detr_ctrl, c='C0', ls='--', lw=1, label='chebychev 13 yr')
ax.plot(AMO_rcp .time/365+ 200,       AMO_filt_detr_rcp , c='C1', ls='--', lw=1)
ax.plot(AMO_lpd .time/365+1350,       AMO_filt_detr_lpd , c='C2', ls='--', lw=1)
ax.plot(AMO_lpi .time/365-1600,       AMO_filt_detr_lpi , c='C3', ls='--', lw=1)

L2, = ax.plot(AMO_ctrl.time/365+1850, lowpass(AMO_ctrl.values, 40)-xr_lintrend(chebychev(AMO_ctrl,13)), c='C0', ls='-' , lw=1.5, label='low pass 40 yr')
ax.plot(AMO_rcp .time/365+ 200,       lowpass(AMO_rcp .values, 40)-xr_lintrend(chebychev(AMO_rcp ,13)), c='C1', ls='-' , lw=1.5)
ax.plot(AMO_lpd .time/365+1350,       lowpass(AMO_lpd .values, 40)-xr_lintrend(chebychev(AMO_lpd ,13)), c='C2', ls='-' , lw=1.5)
ax.plot(AMO_lpi .time/365-1600,       lowpass(AMO_lpi .values, 40)-xr_lintrend(chebychev(AMO_lpi ,13)), c='C3', ls='-' , lw=1.5)

ax.text(1950, .15, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .15, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .15, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .15, 'pre-ind. low' , fontsize=16, color='C3')

ax.legend(handles=[L1, L2], loc=8, ncol=3, fontsize=14, frameon=False)
ax.set_xlabel('time[years]', fontsize=16)
ax.set_ylabel('detrended AMO index [$^\circ$C]'  , fontsize=16)
ax.set_ylim((-.17,.18))
ax.set_xticks(np.arange(0,2400,200))
f.tight_layout()
plt.savefig(f'{path_results}/SST/AMO_detrended_overview')

- comparable period to observations (approx 70 years; Trenberth & Shea)
- much smaller amplitude though (0.05K vs. 0.2K)

In [ ]:
# periodogram
AMOs = [AMO_ctrl, AMO_rcp, AMO_lpd, AMO_lpi]

f, ax = plt.subplots(1, 1, figsize=(8,5), sharey=True)
ax.set_ylabel('spectral power', fontsize=16)

for i in range (4):
    label= ['CTRL', 'RCP', 'LPD', 'LPI'][i]
    f, Pxx = sp.signal.welch(AMOs[i]-xr_lintrend(AMOs[i]), fs=1)
    ax.loglog(1/f, np.sqrt(Pxx), label=label)

ax.tick_params(labelsize=14)
ax.set_xlabel('period [yr]', fontsize=16)
ax.legend(fontsize=14, frameon=False)
    
plt.tight_layout()
# plt.savefig(f'{path_results}/SST/ENSO_spectrum')

In [ ]:
SST_yrly_detr_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc', decode_times=False)

In [ ]:
SST_yrly_detr_ctrl[:,2000,1000].plot()
(10*AMO_filt_detr_ctrl).plot()